In [13]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd
import numba
from numba import njit
import datetime
import datetime
import pandas as pd
import numpy as np
import statsmodels.api as sm
import scipy.optimize as spop
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime as daee
from itertools import combinations
import json
import requests

In [14]:
end = datetime.datetime.now()
start = end - datetime.timedelta(days=365*10)

In [15]:
data = vbt.YFData.pull(
    ["F","GM"],
    start=start,
    end=end,
)   
display(data.shape)

  0%|          | 0/2 [00:00<?, ?it/s]

(2515, 2)

In [16]:
open_price=data.get("Open")["F"]
close_price=data.get("Close")["F"]

In [17]:
def ma_strategy(close_price,window=5,lower_multiple=1,upper_multiple=4):
    signal=np.full(close_price.shape[0],np.nan)
    for x in range(len(close_price)):
        if x >=window:
            ma=np.mean(close_price[x-window:x])
            if close_price[x]<ma*lower_multiple:
                signal[x]=1
            elif close_price[x]>ma*upper_multiple:
                signal[x]=-1
            else:
                signal[x]=0
    return signal

In [18]:
indicator= vbt.IndicatorFactory(
    class_name="Clubbed",
    short_name="Clu",
    input_names=["close_price"],
    param_names=["window","lower_multiple","upper_multiple"],
    output_names=["signal"],
).with_apply_func(
    ma_strategy,
    window=10,
    lower_multiple=1,
    upper_multiple=4,)

In [19]:
results = indicator.run(close_price)
results.signal

Date
2013-10-31 00:00:00-04:00    NaN
2013-11-01 00:00:00-04:00    NaN
2013-11-04 00:00:00-05:00    NaN
2013-11-05 00:00:00-05:00    NaN
2013-11-06 00:00:00-05:00    NaN
                            ... 
2023-10-23 00:00:00-04:00    1.0
2023-10-24 00:00:00-04:00    1.0
2023-10-25 00:00:00-04:00    1.0
2023-10-26 00:00:00-04:00    1.0
2023-10-27 00:00:00-04:00    1.0
Name: F, Length: 2515, dtype: float64

In [20]:
entries= results.signal ==1
exits =results.signal  ==-1

In [21]:
portfolio=vbt.Portfolio.from_signals(close_price,entries,exits,fees=0.001,init_cash=100000) 

In [22]:
portfolio.stats()

c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\generic\stats_builder.py:463: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
c:\Users\jeanc\Anaconda\envs\myenv\lib\site-packages\vectorbtpro\base\wrapping.py:971: UserWarning: Couldn't parse the frequency of index. Pass it as `freq` or define it globally under `settings.wrapping`.
  return self.index_acc.ar

Start                         2013-10-31 00:00:00-04:00
End                           2023-10-27 00:00:00-04:00
Period                                             2515
Start Value                                    100000.0
Min Value                                  33498.264919
Max Value                                 211488.823909
End Value                                  93949.503583
Total Return [%]                              -6.050496
Benchmark Return [%]                          -7.275667
Total Time Exposure [%]                       99.483101
Max Gross Exposure [%]                            100.0
Max Drawdown [%]                              68.787485
Max Drawdown Duration                            1668.0
Total Orders                                          1
Total Fees Paid                                 99.9001
Total Trades                                          1
Win Rate [%]                                        NaN
Best Trade [%]                                  

In [84]:

def cointegrationCacl(data,stock1=0,stock2=1,t_threshold=1.5,window=12):
    signals=np.full(data.shape[0],np.nan)
    
    def unit_root(b):
            a=np.average(data[stock2][t-window:t]-b*data[stock1][t-window:t])
            fair_value=a+b*data[stock1][t-window:t]
            diff=np.array(fair_value-data[stock2][t-window:t])
            diff_diff=diff[1:]-diff[:-1]
            
            reg=sm.OLS(diff_diff, diff[:-1])
            res=reg.fit()
            
            return (res.params[0]/res.bse[0])
    x=0
    for t in range(window,len(data)):
        x+=1

        print(data.shape)
        
        res1=spop.minimize(unit_root,data[t][stock2]/data[t][stock1],method='Nelder-Mead')
        t_opt=res1.fun
        b_opt=float(res1.x)
        a_opt=np.average(data[stock2][t-window:t]-b_opt*data[stock1][t-window:t])
        fair_value=a_opt+b_opt*data[stock1][t]

        if t==window:
            old_signal=0
        if t_opt > t_threshold:
            signal=0
        else:
            signal=np.sign(fair_value-data[stock2][t])

        t_s=np.append(t_s,t_opt)

        
        
        
        tradeloglist=["no trading","long position on "+ stock2+" and short position on "+stock1,"long position on "+ stock1 +" and short position on "+stock2]
        if signal==0:
            signals[t]=0
        elif signal==1:
            signals[t]=1
        else:
            signals[t]=-1
        tradingpair=str(data.index[t])
        old_signal=signal
    return signals


In [85]:
open_price=data.get("Open")
close_price=data.get("Close")
print(open_price)

symbol                             F         GM
Date                                           
2013-10-31 00:00:00-04:00  10.854520  28.384647
2013-11-01 00:00:00-04:00  10.823129  28.377012
2013-11-04 00:00:00-05:00  10.653624  28.628949
2013-11-05 00:00:00-05:00  10.603401  28.460985
2013-11-06 00:00:00-05:00  10.772904  28.430458
...                              ...        ...
2023-10-23 00:00:00-04:00  11.550000  29.330000
2023-10-24 00:00:00-04:00  11.500000  28.389999
2023-10-25 00:00:00-04:00  11.350000  28.549999
2023-10-26 00:00:00-04:00  11.910000  29.809999
2023-10-27 00:00:00-04:00  10.930000  28.490000

[2515 rows x 2 columns]


In [86]:
indicator= vbt.IndicatorFactory(
    class_name="Cointegration",
    short_name="Coint",
    input_names=["close_price"],
    param_names=["window","t_treshold"],
    output_names=["signal"],
).with_apply_func(
    cointegrationCacl,
    window=10,
    t_treshold=1.5,
    )

In [87]:
display(close_price)
print(close_price["F"][200])

symbol,F,GM
Date,,
2013-10-31 00:00:00-04:00,10.741517,28.209057
2013-11-01 00:00:00-04:00,10.603402,28.544970
2013-11-04 00:00:00-05:00,10.672459,28.606047
2013-11-05 00:00:00-05:00,10.728960,28.315933
2013-11-06 00:00:00-05:00,10.615956,27.934221
...,...,...
2023-10-23 00:00:00-04:00,11.480000,29.219999
2023-10-24 00:00:00-04:00,11.390000,28.559999
2023-10-25 00:00:00-04:00,11.540000,29.010000


11.151007652282715


C:\Users\jeanc\AppData\Local\Temp\ipykernel_29560\1603826694.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(close_price["F"][200])


In [88]:
results = indicator.run(close_price)

(2515, 2)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices